In [6]:
# Import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression

In [7]:
# Create SQLite connection
happiness_path = Path('../Resources/HappinessIndexScore.sqlite')
engine = create_engine(f'sqlite:///{happiness_path}')

In [8]:
df_gdp = pd.read_sql('SELECT * FROM final_output', con=engine)
df_gdp.head()

,country,region,ladder_score,logged_GDP_per_capita,social_support,healthy_life_expectancy,freedom_life_choices,generosity,perceptions_corruption,population_density,unemployment_rate,median_age,gini_coefficient,avg_temperature,lt_alcohol_per_capita
0,Finland,Europe,7.804,10.792,0.969,71.150,0.961,-0.019,0.182,16.60,7.16,43.2,27.7,3.24,8.23
1,Denmark,Europe,7.586,10.962,0.954,71.250,0.934,0.134,0.196,138.00,5.14,42.2,27.7,9.77,9.16
2,Iceland,Europe,7.530,10.896,0.983,72.050,0.936,0.211,0.668,3.50,3.56,37.8,26.1,2.11,7.72
3,Israel,Middle East,7.473,10.639,0.943,72.697,0.809,-0.023,0.708,412.24,3.39,30.1,38.6,20.23,3.07
4,Netherlands,Europe,7.403,10.942,0.930,71.550,0.887,0.213,0.379,420.38,3.56,42.2,29.2,11.72,8.23


In [9]:
# Create a scatter plot of gdp versus the ladder score information
gdp_plot = df_gdp.hvplot.scatter(
    x="logged_GDP_per_capita",
    y="ladder_score",
    title="Ladder_score vs. gdp per capita"
)
gdp_plot

:Scatter   [logged_GDP_per_capita]   (ladder_score)

In [10]:
# Reformat data of the independent variable X as a single-column array
X = df_gdp["logged_GDP_per_capita"].values.reshape(-1, 1)

# Display sample data
X[:5]

array([[10.792],
       [10.962],
       [10.896],
       [10.639],
       [10.942]])

In [11]:
# The shape of X is 30 samples, with a single feature (column)
X.shape

(123, 1)

In [12]:
# Create an array for the dependent variable y
y = df_gdp["ladder_score"]

In [13]:
# Create a model with scikit-learn
model = LinearRegression()

In [14]:
# Fit the data into the model
model.fit(X, y)

LinearRegression()

In [15]:
# Display the slope
print(f"Model's slope: {model.coef_}")

Model's slope: [0.76152706]


In [16]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's formula: y = -1.63260618863929 + 0.7615270646405502X


In [17]:

print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}")

y_7 = model.intercept_ + model.coef_[0] 

# Display the prediction
print(f"{y_7:.2f}")

Model's formula: y = -1.63260618863929 + 0.7615270646405502
-0.87


In [18]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [19]:
# Create a copy of the original data
df_happiness_predicted = df_gdp.copy()

# Add a column with the predicted ladder score values
df_happiness_predicted["ladder_score_predicted"] = predicted_y_values

# Display sample data
df_happiness_predicted.head()

,country,region,ladder_score,logged_GDP_per_capita,social_support,healthy_life_expectancy,freedom_life_choices,generosity,perceptions_corruption,population_density,unemployment_rate,median_age,gini_coefficient,avg_temperature,lt_alcohol_per_capita,ladder_score_predicted
0,Finland,Europe,7.804,10.792,0.969,71.150,0.961,-0.019,0.182,16.60,7.16,43.2,27.7,3.24,8.23,6.585794
1,Denmark,Europe,7.586,10.962,0.954,71.250,0.934,0.134,0.196,138.00,5.14,42.2,27.7,9.77,9.16,6.715253
2,Iceland,Europe,7.530,10.896,0.983,72.050,0.936,0.211,0.668,3.50,3.56,37.8,26.1,2.11,7.72,6.664993
3,Israel,Middle East,7.473,10.639,0.943,72.697,0.809,-0.023,0.708,412.24,3.39,30.1,38.6,20.23,3.07,6.469280
4,Netherlands,Europe,7.403,10.942,0.930,71.550,0.887,0.213,0.379,420.38,3.56,42.2,29.2,11.72,8.23,6.700023


In [20]:
# Create a line plot of gdp versus the predicted ladder score values
best_fit_line = df_happiness_predicted.hvplot.line(
    x = "logged_GDP_per_capita",
    y = "ladder_score_predicted",
    color = "red"
)
best_fit_line

:Curve   [logged_GDP_per_capita]   (ladder_score_predicted)

In [21]:
gdp_plot * best_fit_line

:Overlay
   .Scatter.I :Scatter   [logged_GDP_per_capita]   (ladder_score)
   .Curve.I   :Curve   [logged_GDP_per_capita]   (ladder_score_predicted)

In [22]:
# Import relevant metrics from scikit-learn
from sklearn.metrics import mean_squared_error, r2_score

# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.6249707348153062.
The r2 is 0.6249707348153062.
The mean squared error is 0.49570923968157665.
The root mean squared error is 0.7040662182505113.
The standard deviation is 1.1496904595668735.
